# Assignment 03 - AE21B105
This notebook will be used as the base version of testing the code written and for the sweeps later on. Then once finalized and all good this will be transfered to a script with command line arguments. Lets begin !!!

In [ ]:
# Importing the necessary libraries needed
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms.functional as F
import lightning as L
from lightning.pytorch import Trainer
from torch.utils.data import DataLoader, Subset
from lightning.pytorch.loggers import WandbLogger
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import os

In [ ]:
# Data preparation
